In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import load_model
from cbow_model import *

In [2]:
# Parameters
window_size = 2 
window_size_corpus = window_size*2

# Set numpy seed for reproducible results
np.random.seed(42)

In [3]:
#cbow_model = cbow_model(vocab_size = V, embedding_dim = 100 , window_size = window_size)
cbow_model = load_model('my_model.h5')
cbow_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            255700    
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2557)              258257    
                                                                 
Total params: 513,957
Trainable params: 513,957
Non-trainable params: 0
_________________________________________________________________


In [4]:
file_name = 'ALICE.txt'
corpus = open(file_name).readlines()

# Remove sentences with fewer than 3 words
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# Remove punctuation in text and fit tokenizer on entire corpus
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)

# Convert text to sequence of integer values
corpus = tokenizer.texts_to_sequences(corpus)
n_samples = sum(len(s) for s in corpus) # Total number of words in the corpus
V = len(tokenizer.word_index) + 1 # Total number of unique words in the corpus
my_dict = tokenizer.word_index

vocab = []
for key, value in my_dict.items():
    vocab.append(key)

print(len(vocab))

2556


In [5]:
file_name = 'ALICE.txt'
corpus_lst = open(file_name).readlines()
# Remove sentences with fewer than 3 words
corpus_lst = [sentence for sentence in corpus_lst if sentence.count(" ") >= 2]

In [6]:
all_words =" ".join(corpus_lst).split()

In [7]:
# Define the vocabulary
vocabulary = vocab

# Define a context of words
context = 'another wer wer  wer went'.split()
context

['another', 'wer', 'wer', 'wer', 'went']

In [8]:
# Convert the context to a vector
context_vector = np.zeros((len(context),))
for word in context:
    if word in vocabulary:
        context_vector[context.index(word)] += 1
# Normalize the context vector
context_vector /= np.linalg.norm(context_vector)
context_vector

array([0.70710678, 0.        , 0.        , 0.        , 0.70710678])

In [9]:
for n in context_vector:
    if n != 0 :
        print(n)

0.7071067811865475
0.7071067811865475


In [10]:
n = np.array([[1, 2, 3, 1, 2, 3]])
n.reshape(3,2)

array([[1, 2],
       [3, 1],
       [2, 3]])

In [11]:
# Reshape the context vector to have a shape of (1, len(vocabulary))
context_vector = context_vector.reshape(len(vocab) , len(context))
context_vector

ValueError: cannot reshape array of size 5 into shape (2556,5)

In [ ]:
context_vector.shape

(1, 5)

In [ ]:
# Predict the target word
prediction = cbow_model.predict(context_vector)

# Get the index of the predicted word
predicted_index = np.argmax(prediction)

# Get the predicted word from the vocabulary
predicted_word = vocabulary[predicted_index]

print("The predicted next word is:", predicted_word)

ValueError: in user code:

    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users

In [ ]:
model.get_layer('embedding').get_weights()

NameError: name 'model' is not defined

In [ ]:
import pandas as pd

In [ ]:
x = cbow_model.get_layer('embedding').get_weights()[0]
x.shape

(2557, 100)

In [ ]:
pd.DataFrame(x).T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.28074,-0.898299,-0.114706,0.358534,-0.505169,0.03178,0.390315,0.155193,1.063202,0.78178,...,0.553495,0.258711,0.472479,0.351076,-0.627747,-0.853452,-0.049117,0.187064,-0.875002,0.40091


In [ ]:
# Save embeddings for vectors of length 50, 150 and 300 using cbow model
weights = cbow_model.get_weights()

# Get the embedding matrix
embedding = weights[0]

In [ ]:
embedding.shape

(2557, 100)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
def embed(word, embedding =cbow_model.get_weights()[0] , vocab_size=V, tokenizer=tokenizer):
    """ Embed a word by getting the one hot encoding and taking the dot product of this vector with the 
        embedding matrix 'word' = string type
    """
    # get the index of the word from the tokenizer, i.e. convert the string to it's corresponding integer in the vocabulary
    int_word = tokenizer.texts_to_sequences([word])[0]
    # get the one-hot encoding of the word
    bin_word = to_categorical(int_word, V)
    return np.dot(bin_word, embedding)

In [ ]:
len(embed('alice')[0])

100

In [ ]:
# Define a context of words
context = 'another moment down went'.split()
context

['another', 'moment', 'down', 'went']

In [ ]:
#Convert the context to a vector
context_vector = np.zeros((len(vocab),))
for word in context:
    if embed(word) in embedding:
        context_vector[vocab.index(word)] += 1

# Normalize the context vector
context_vector /= np.linalg.norm(context_vector)

# Reshape the context vector to have a shape of (1, len(vocabulary))
context_vector = context_vector.reshape(1, len(vocab))
context_vector
# Predict the target word
prediction = cbow_model.predict(context_vector)[0]

# Get the index of the predicted word
predicted_index = np.argmax(prediction)

# Get the predicted word from the vocabulary
predicted_word = vocab[predicted_index]

print("The predicted next word is:", predicted_word)

1/1 [==============================] - 0s 38ms/step
The predicted next word is: people


In [ ]:
for w in vocab:
    if w =='In':
        print('w')